In [50]:
#gini impurity : 0 <= 1 - p_survived^2 - p_perish^2 <= 0.5
# model validation : percentage <  another : (p_c + p_uc) /2 
#research engineer 

import numpy as np
import pandas as pd

# datetime에 해당하는 컬럼 모두 적용하기 위해 리스트 형식으로 넣는다
train = pd.read_csv("data/train.csv", parse_dates=["datetime"]) 
# 날짜데이터 호출방법 : train["datetime"].dt.year (month,day,hour,minute,second)
# 날짜데이터의 
# pandas, python 날짜 코딩방법 공부 
# one hot encoding : 1,2,3,4 - 매출액 : 1: 100 -> 2: 200 ?

train.head() 

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [51]:
test = pd.read_csv("data/test.csv",parse_dates=["datetime"])
print(train.shape, test.shape) # col diff : casual 	registered 	count
test.head()

(10886, 12) (6493, 9)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


## Train

In [52]:
feature_names = ["season","holiday","workingday","weather","temp","atemp","humidity","windspeed"]
feature_names

['season',
 'holiday',
 'workingday',
 'weather',
 'temp',
 'atemp',
 'humidity',
 'windspeed']

In [53]:
#train data
X_train = train[feature_names]
print(X_train.shape)
Y_train = train["count"]
print(Y_train.shape)

#test data
X_test = test[feature_names]
print(X_test.shape)

(10886, 8)
(10886,)
(6493, 8)


from sklearn.tree import DecisionTreeRegressor # 회귀분석
model = DecisionTreeRegressor()
model

In [65]:
from sklearn.ensemble import RandomForestRegressor # 회귀분석
model = RandomForestRegressor()
model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

## score

In [66]:
#from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import cross_val_score

#Hold-out validation (고정)   , Cross validation(순환)
score = cross_val_score(model, X_train, Y_train, cv=20,scoring="neg_mean_absolute_error").mean()
#principal


print("Score = {0:.5f}".format(score))
# 의사결정나무 : 153 -> 랜덤포레스트 : 129
# 느리고 병렬처리 안되는 svm 보다 randomeForeset 선호
# 66% 만 추출해서 모델만들고, 모델의 투표로 예측한다


Score = -129.87062


 1. actual = 100개
 2. model A = 120
 3. model B = 70

regression 모델 측정방법 (error = 차이)
1) |p-a|    mean absolute error 
2) (p-a)^2  mean squared error   # error에 대한 penalty를 크게 한다.
3) root((p-a)^2) root mean squared error
4) root((log(p+1)-log(a+1)^2) RMLSE #사소한 error는 크게하고 지나친 error는 작게한다

 

In [67]:
model.fit(X_train,Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [68]:
prediction=model.predict(X_test)
print(prediction.shape)
prediction[0:10] #array type

(6493,)


array([ 204.3       ,  106.05      ,  106.05      ,   35.7       ,
         35.7       ,   43.7       ,  143.3       ,   55.6       ,
         48.2       ,  124.40833333])

In [69]:
submission = pd.read_csv("data/sampleSubmission.csv")
submission["count"]=prediction
print(submission.shape)
submission.head()

(6493, 2)


,datetime,count
0,2011-01-20 00:00:00,204.30
1,2011-01-20 01:00:00,106.05
2,2011-01-20 02:00:00,106.05
3,2011-01-20 03:00:00,35.70
4,2011-01-20 04:00:00,35.70


In [58]:
submission.to_csv("baseline-script.csv",index=False) # index 없애고 저장하기 